In [1]:
import copy
import time
import threading
import pandas as pd
import numpy as np
import more_itertools
import csv
import random
import itertools
pd.options.mode.chained_assignment = None  # default = 'warn'

In [2]:
product_rules = pd.read_csv('Data/master_rules.csv')
# product_rules.drop(columns=['Unnamed: 0'], inplace=True)
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,frozenset({'Instant food products'}),frozenset({'soda'}),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,frozenset({'Instant food products'}),frozenset({'whole milk'}),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,frozenset({'UHT-milk'}),frozenset({'whole milk'}),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,frozenset({'abrasive cleaner'}),frozenset({'rolls/buns'}),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,frozenset({'artif. sweetener'}),frozenset({'other vegetables'}),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121


In [3]:
with open('Data/names.csv') as f:
    read = csv.reader(f)
    names = list(read)

In [4]:
print(names)

[['Instant food products', 'UHT-milk', 'abrasive cleaner', 'artif. sweetener', 'baby cosmetics', 'bags', 'baking powder', 'bathroom cleaner', 'beef', 'berries', 'beverages', 'bottled beer', 'bottled water', 'brandy', 'brown bread', 'butter', 'butter milk', 'cake bar', 'candles', 'candy', 'canned beer', 'canned fish', 'canned fruit', 'canned vegetables', 'cat food', 'cereals', 'chewing gum', 'chicken', 'chocolate', 'chocolate marshmallow', 'citrus fruit', 'cleaner', 'cling film/bags', 'cocoa drinks', 'coffee', 'condensed milk', 'cooking chocolate', 'cookware', 'cream', 'cream cheese ', 'curd', 'curd cheese', 'decalcifier', 'dental care', 'dessert', 'detergent', 'dish cleaner', 'dishes', 'dog food', 'domestic eggs', 'female sanitary products', 'finished products', 'fish', 'flour', 'flower (seeds)', 'flower soil/fertilizer', 'frankfurter', 'frozen chicken', 'frozen dessert', 'frozen fish', 'frozen fruits', 'frozen meals', 'frozen potato products', 'frozen vegetables', 'fruit/vegetable jui

In [5]:
print(len(names[0]))

167


In [6]:
product_rules.dtypes

antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
dtype: object

In [7]:
def fix_frozen_set(x):
    """
    :param x:
    :return: takes panda dataframe and reformats antecedents and consequents as Frozensets for correct querying
    """
    beginning = x.find("'") + 1
    end = x.rfind("'")
    data = x[beginning: end]
    data_remove_apostrophes = data.replace("'", '')
    new_data = data_remove_apostrophes.split(',')
    new_data_stripped = []
    for i in range(len(new_data)):
        new_data_stripped.append(copy.copy(new_data[i].strip()))
    if len(new_data_stripped) == 1:
        return frozenset(new_data_stripped)
    else:
        return frozenset([x for x in new_data_stripped])


In [8]:
product_rules['antecedents'] = product_rules['antecedents'].apply(fix_frozen_set)
product_rules['consequents'] = product_rules['consequents'].apply(fix_frozen_set)
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121


In [9]:
product_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121
...,...,...,...,...,...,...,...,...,...
24247,"(soda, bottled water, other vegetables, rolls/...",(root vegetables),0.003849,0.230690,0.002566,0.666667,2.889878,0.001678,2.307929
24248,"(root vegetables, bottled water, other vegetab...",(soda),0.003336,0.313318,0.002566,0.769231,2.455112,0.001521,2.975622
24249,"(root vegetables, soda, other vegetables, roll...",(bottled water),0.003336,0.213754,0.002566,0.769231,3.598670,0.001853,3.407065
24250,"(root vegetables, soda, bottled water, other v...",(rolls/buns),0.003336,0.349756,0.002566,0.769231,2.199334,0.001399,2.817723


In [80]:
# def clean_up(factored_answer):
#     if len(factored_answer) == 1:
#         return factored_answer
#     to_check = copy.deepcopy(factored_answer)
#     index = 0
#     while index < len(to_check):
#         if to_check[index].find(','):
#             to_split = copy.copy(to_check[index].split(','))
#             to_check.pop(index)
#             for j in range(len(to_split)):
#                 to_check.append(copy.copy(to_split[j]))
#             index = 0
#         else:
#             index += 1
#             continue
#     return set(to_check)

def recommend(z):  # input is list of lists
    print("Z contains: ", z)
    print()
    print("Length of Z is: ",len(z))
    print()
    global product_rules
    master_query = None
    while True:
        try:
            master_query = product_rules[product_rules['antecedents'] == {z[0], z[1], z[2], z[3], z[4]}]
        except IndexError:
            print("Query after not having 5 entries is: ",master_query)
            print("Type of query is: ", type(master_query))
            print()
            pass
        if master_query is not None:
            break
        try:
            print("Made it to second try statement!!\n")
            master_query = product_rules[product_rules['antecedents'] == {z[0], z[1], z[2], z[3]}]
        except IndexError:
            pass
        if master_query is not None:
            break
        try:
            print("Made it to the third try statement!!\n")
            master_query = product_rules[product_rules['antecedents'] == {z[0], z[1], z[2]}]
        except IndexError:
            pass
        if master_query is not None:
            break
        try:
            print("Made it to the fourth try statement!!\n")
            master_query = product_rules[product_rules['antecedents'] == {z[0], z[1]}]
        except IndexError:
            pass
        if master_query is not None:
            break
        try:
            print("Made it to the last try statement!!\n")
            master_query = product_rules[product_rules['antecedents'] == {z[0]}]
        except IndexError:
            return "OOPS!"
        if master_query is None:
            return None
    master_query.sort_values(by=['support','confidence','lift'],inplace=True,ascending=(False,False,False), ignore_index=True)
    # maybe update possible graph here ???
    master_query['consequents'] = master_query['consequents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
    results = list(master_query['consequents'])
    print(len(results))
    print(results)
    if len(results) == 0:
        return None
    else:
        return set(results)
    # if len(results) == 1:
    #     print("GOT CAUGHT IN THE IF STATEMENT!!")
    #     return results
    # print('MADE IT PAST THE IF STATEMENT!!!')
    # factored_answer = set()
    # for value in range(len(results)):
    #     factored_answer.add(copy.copy(results[value]))
    # print("Factored answer: ",factored_answer)
    # print(len(factored_answer))
    # factored_answer_after_cleanup = clean_up(factored_answer)
    # return factored_answer_after_cleanup
    # df[df['antecedents'] == {'rolls/buns', 'Instant food products'}]

In [81]:
# def synchronized(func):
#     func.__lock__ = threading.Lock()
#
#     def synced_func(*args, **kws):
#         with func.__lock__:
#             return func(*args, **kws)
#     return synced_func

In [102]:
def start_here(x):
    master_search = []
    for j in range(1,6):
        for element in itertools.combinations(x, j):
            master_search.append(list(element))
    print("Length of master search is: ", len(master_search))
    while True:
        try:
            print("GOING INTO RECOMMEND!!!\n\n")
            response = recommend(master_search[-1])
            print("LEFT RECOMMEND!!!\n\n")
        except IndexError:
            return 'SOMETHING WENT WRONG!!!'
        if response is None or response == "OOPS!":
            master_search.pop()
            print("Length of Master Search is: ", len(master_search))
            print("Master search contains: ", master_search)
            response = None
            continue
        elif response == 0: # this becomes else statement once code is added TODO
            pass
            # add code to check if item returned is in list already
        else:
            print("MAKES IT TO FINAL ELSE!!! \n\n\n")
            # add code to check if items already in inventory... TODO
            return response
    # print("Master Search Query Contains: ", master_search)
    # print('Length of Master Search Query: ', len(master_search))
    # print('\n\n\n')
    # print('\n\n\n')
    # print('Cleaned up version is: ', response)
    # print("Length is: ", len(response))
    # return response

In [103]:
list_to_check_5 = []
for i in range(5):
    random_index = random.randrange(len(names[0]))
    list_to_check_5.append(copy.copy(names[0][random_index]))

In [104]:
print(list_to_check_5)

['sugar', 'whisky', 'spices', 'cooking chocolate', 'long life bakery product']


In [105]:
ans5 = start_here(list_to_check_5)

Length of master search is:  31
GOING INTO RECOMMEND!!!


Z contains:  ['sugar', 'whisky', 'spices', 'cooking chocolate', 'long life bakery product']

Length of Z is:  5

0
[]
LEFT RECOMMEND!!!


Length of Master Search is:  30
Master search contains:  [['sugar'], ['whisky'], ['spices'], ['cooking chocolate'], ['long life bakery product'], ['sugar', 'whisky'], ['sugar', 'spices'], ['sugar', 'cooking chocolate'], ['sugar', 'long life bakery product'], ['whisky', 'spices'], ['whisky', 'cooking chocolate'], ['whisky', 'long life bakery product'], ['spices', 'cooking chocolate'], ['spices', 'long life bakery product'], ['cooking chocolate', 'long life bakery product'], ['sugar', 'whisky', 'spices'], ['sugar', 'whisky', 'cooking chocolate'], ['sugar', 'whisky', 'long life bakery product'], ['sugar', 'spices', 'cooking chocolate'], ['sugar', 'spices', 'long life bakery product'], ['sugar', 'cooking chocolate', 'long life bakery product'], ['whisky', 'spices', 'cooking chocolate'], ['whisky',

In [106]:
print(ans5)

{'rolls/buns', 'other vegetables', 'whole milk'}


In [107]:
print(type(ans5))

<class 'set'>


In [108]:
print(ans5.pop())

rolls/buns


In [109]:
print(len(ans5))

2


In [110]:
# print(type(ans5.pop()))

In [111]:
list_to_check_4 = []
for i in range(4):
    random_index = random.randrange(len(names[0]))
    list_to_check_4.append(copy.copy(names[0][random_index]))
print(list_to_check_4)

['liquor', 'rum', 'sugar', 'beverages']


In [112]:
ans4 = start_here(list_to_check_4)

Length of master search is:  15
GOING INTO RECOMMEND!!!


Z contains:  ['liquor', 'rum', 'sugar', 'beverages']

Length of Z is:  4

Query after not having 5 entries is:  None
Type of query is:  <class 'NoneType'>

Made it to second try statement!!

0
[]
LEFT RECOMMEND!!!


Length of Master Search is:  14
Master search contains:  [['liquor'], ['rum'], ['sugar'], ['beverages'], ['liquor', 'rum'], ['liquor', 'sugar'], ['liquor', 'beverages'], ['rum', 'sugar'], ['rum', 'beverages'], ['sugar', 'beverages'], ['liquor', 'rum', 'sugar'], ['liquor', 'rum', 'beverages'], ['liquor', 'sugar', 'beverages'], ['rum', 'sugar', 'beverages']]
GOING INTO RECOMMEND!!!


Z contains:  ['rum', 'sugar', 'beverages']

Length of Z is:  3

Query after not having 5 entries is:  None
Type of query is:  <class 'NoneType'>

Made it to second try statement!!

Made it to the third try statement!!

0
[]
LEFT RECOMMEND!!!


Length of Master Search is:  13
Master search contains:  [['liquor'], ['rum'], ['sugar'], ['bever

In [113]:
print(ans4)

{'sausage', 'other vegetables', 'whole milk'}


In [114]:
list_to_check_3 = []
for i in range(3):
    random_index = random.randrange(len(names[0]))
    list_to_check_3.append(copy.copy(names[0][random_index]))
print(list_to_check_3)

['tropical fruit', 'mustard', 'white wine']


In [115]:
ans3 = start_here(list_to_check_3)

Length of master search is:  7
GOING INTO RECOMMEND!!!


Z contains:  ['tropical fruit', 'mustard', 'white wine']

Length of Z is:  3

Query after not having 5 entries is:  None
Type of query is:  <class 'NoneType'>

Made it to second try statement!!

Made it to the third try statement!!

0
[]
LEFT RECOMMEND!!!


Length of Master Search is:  6
Master search contains:  [['tropical fruit'], ['mustard'], ['white wine'], ['tropical fruit', 'mustard'], ['tropical fruit', 'white wine'], ['mustard', 'white wine']]
GOING INTO RECOMMEND!!!


Z contains:  ['mustard', 'white wine']

Length of Z is:  2

Query after not having 5 entries is:  None
Type of query is:  <class 'NoneType'>

Made it to second try statement!!

Made it to the third try statement!!

Made it to the fourth try statement!!

0
[]
LEFT RECOMMEND!!!


Length of Master Search is:  5
Master search contains:  [['tropical fruit'], ['mustard'], ['white wine'], ['tropical fruit', 'mustard'], ['tropical fruit', 'white wine']]
GOING INTO 

In [116]:
print(ans3)

{'yogurt'}


In [117]:
list_to_check_2 = []
for i in range(2):
    random_index = random.randrange(len(names[0]))
    list_to_check_2.append(copy.copy(names[0][random_index]))
print(list_to_check_2)

['pickled vegetables', 'citrus fruit']


In [118]:
ans2 = start_here(list_to_check_2)

Length of master search is:  3
GOING INTO RECOMMEND!!!


Z contains:  ['pickled vegetables', 'citrus fruit']

Length of Z is:  2

Query after not having 5 entries is:  None
Type of query is:  <class 'NoneType'>

Made it to second try statement!!

Made it to the third try statement!!

Made it to the fourth try statement!!

1
['whole milk']
LEFT RECOMMEND!!!


MAKES IT TO FINAL ELSE!!! 





In [119]:
print(ans2)

{'whole milk'}


In [120]:
list_to_check_1 = []
for i in range(1):
    random_index = random.randrange(len(names[0]))
    list_to_check_1.append(copy.copy(names[0][random_index]))
print(list_to_check_1)

['soups']


In [121]:
ans1 = start_here(list_to_check_1)

Length of master search is:  1
GOING INTO RECOMMEND!!!


Z contains:  ['soups']

Length of Z is:  1

Query after not having 5 entries is:  None
Type of query is:  <class 'NoneType'>

Made it to second try statement!!

Made it to the third try statement!!

Made it to the fourth try statement!!

Made it to the last try statement!!

Query after not having 5 entries is:      antecedents   consequents  antecedent support  consequent support  \
112     (soups)        (soda)            0.012317            0.313318   
115     (soups)  (whole milk)            0.012317            0.458045   

      support  confidence      lift  leverage  conviction  
112  0.006159    0.500000  1.595823  0.002299    1.373364  
115  0.006415    0.520833  1.137080  0.000773    1.131037  
Type of query is:  <class 'pandas.core.frame.DataFrame'>

2
['whole milk', 'soda']
LEFT RECOMMEND!!!


MAKES IT TO FINAL ELSE!!! 





In [122]:
print(ans1)

{'soda', 'whole milk'}


In [223]:
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121


In [165]:
product_rules.to_csv('Data/master_rules.csv', index=False)

In [225]:
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121
